In [27]:
# -*- coding: utf-8 -*-
# @Time    : 2025/04/03 
import os
import re
import emoji
from zhon.hanzi import punctuation as cn_punc
import json
import glob
from langid import classify
import jieba 

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
# Load the MeetFresh user dictionary
jieba.load_userdict("MF_dict.txt")

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Han\AppData\Local\Temp\jieba.cache
Loading model cost 1.117 seconds.
Prefix dict has been built successfully.


In [29]:
# Define the constants

core_keywords = {
    '鲜芋仙', 'Meet Fresh', 'MeetFresh', '台湾美食', '甜品', 
    '芋圆', 'taro', '仙草', 'grass jelly', '奶茶', 'milk tea',
    '豆花', 'tofu pudding', '奶刨冰', 'milked shaved ice',
    '红豆汤', 'purple rice soup', '紫米粥', 'red bean soup',
    '2001 Coit Rd', 'Park Pavillion Center', '(972) 596-6088'
}



In [39]:
cont = pd.read_json('..\..\Data\processed\contents_cooked.json')

In [40]:
cont.head(2)

,note_id,user_id,title,note_body,tag_list,image_count,content_type_video,hot_note,post_time,last_update_time,scraped_time,liked_count,collected_count,comment_count,share_count,interaction_count,text_length
0,67d0d605000000002903d86a,576d3bde82ec3952ff40c5e1,有没有和我一样【吃茶三千】一喝一个不吱声的,连地址都不想写了[笑哭R]\n虽然环境还不错，颜值也不错\n但我感觉自己完全拔草了\n尝了m...,"达拉斯网红奶茶,达拉斯奶茶,达拉斯,达拉斯生活,达拉斯美食,达拉斯周边,达拉斯探店",1,0,0,2025-03-12 00:32:05,2025-03-12 00:32:06,2025-03-12 00:55:08.175,5,2,10,3,17,300
1,67c8960f000000002503ff5f,576d3bde82ec3952ff40c5e1,【达拉斯·玩】超治愈系手工体验！MUMU Garden,"📍301 W Parker Rd #208[话题]#, Plano, TX 75023（快乐...","208,达拉斯生活,达拉斯探店,达拉斯,达拉斯吃喝玩乐,达拉斯手工,达拉斯周末,达拉斯周边,...",14,0,0,2025-03-05 18:21:03,2025-03-05 18:21:04,2025-03-12 00:55:08.248,47,23,7,37,77,847


In [41]:
# combine the title and note_body into a single string
def process_text(note):
    return note['title'] + ' ' + note['note_body']

# Apply the function to the DataFrame
cont['text'] = cont.apply(process_text, axis=1).astype(str)


In [ ]:
# slice the 1st row of cont
cont.iloc[11]

'[达拉斯.吃]刘一手,目前火锅届性价比天花板 📍2001 Coit Rd Ste 137D, Plano, TX 75075 听说新团队接手了达拉斯刘一手,好吃又实惠,特意和朋友跑去瞅瞅什么情况 感觉确实诚不欺我!虽然新菜单还没做出来,但一些规则已经改为和休斯顿一样了,小朋友5岁以下免费,5-12岁收儿童自助餐费十几刀,然后目前小料台依旧免费(卤牛肉/猪耳朵/小馒头/麻球/点心/水果/冰粉吃到饱) 最重要的是锅底据说由集团负责人亲自监督味道,辣锅底料不再是加拿大运来的国际版,而是和休斯顿一样从国内直接进口的原版,确实好吃不少(就是我只敢点微辣[笑哭R]) 每次除了肉片,我家必点麻辣牛,片片竹笋,鱼籽福袋,鱼籽虾滑,黑鱼片,午餐肉,西洋菜,山药,藤椒千层肚(别点错成要钱的大刀毛肚[笑哭R]他们说新菜单会特别标注),手工面和爆浆糍粑(这个$4.95值!),真的都很好吃!!特别推荐不辣的寿喜锅,味道很足很日系,健康感这就来了 因为老公巨能吃肉,去点单的火锅店随便吃吃,小费后每次都要人均$7,80[哭惹R]在刘一手,两个人工作日晚上+鸳鸯锅底,$27.95+$13.95,单人加税只要$37.8!!! 而且人越多人均越低,四个人就能$32 (但选小费时请记得一定不要不好意思,可以custom输入你认为值得的价值,厚face如我)加上味道真是已经挺好吃了,非常对得起价格[飞吻R] 店员说这几天还有免费血浆袋酸梅汤赠送,可惜我太饱了没来一袋 明天的万圣节穿custume打卡,可以四人同行一人免单![仙女R][仙女R][仙女R]感兴趣的赶紧冲冲冲! PS:饭后别忘了要bluebell冰淇淋哦~这丝滑的羊毛绝对得薅[大笑R] #北美刘一手[话题]# #达拉斯火锅[话题]# #达拉斯美食[话题]# #达拉斯生活[话题]# #达拉斯[话题]# #达拉斯刘一手[话题]# #达拉斯探店[话题]# #达拉斯周边[话题]# #达拉斯周末[话题]# #达拉斯吃喝玩乐[话题]#'

中文文本清洗黄金四步法

In [34]:
def fullwidth_to_halfwidth(text):
    """全角转半角（保留￥符号）"""
    translation_table = str.maketrans({
        '！': '!', '“': '"', '”': '"', '‘': "'", '’': "'",
        '、': ',', '，': ',', '；': ';', '：': ':', '？': '?',
        '《': '<', '》': '>', '【': '[', '】': ']', '·': '.',
        '～': '~', '—': '-', '（': '(', '）': ')', '　': ' '
    })
    return text.translate(translation_table)

def normalize_punctuation(text):
    """符号标准化（保留emoji, retain 字符的位置信息但牺牲了效率, 之后可以考虑优化"""
    # 定义保留符号集（新增%$￥）
    keep_symbols = {"'", '"', ',', '.', ':', ';', '!', '?', '-', 
                   '(', ')', '<', '>', '[', ']', '&', '#', '@',
                   '%', '$', '￥', '/', '=', '+', '~', '^'}
    
    # 字符级处理, 
    cleaned_chars = []
    for char in text:
        # 保留条件：字母数字/汉字/keep_symbols/emoji
        if (char.isalnum() or
            '\u4e00' <= char <= '\u9fff' or
            char in keep_symbols or
            emoji.is_emoji(char)):
            cleaned_chars.append(char)
        else:
            cleaned_chars.append(' ')
    
    return ''.join(cleaned_chars)

def remove_urls(text):
    """适配中文域名的URL移除"""
    url_pattern = re.compile(
        r'(?:https?://)?(?:[a-zA-Z0-9\u4e00-\u9fff-]+\.)+[a-zA-Z]{2,}'
        r'(?:/\S*)?(?:\?\S*)?(?:#\S*)?',
        flags=re.IGNORECASE
    )
    return url_pattern.sub('', text)

def basic_clean(text):
    """
    对文本进行基础层清洗，包括去除HTML标签、URL、特殊符号处理, 空格标准化等。
    1. 移除HTML标签
    2. 移除URL（适配中文域名）
    3. 处理特殊符号（保留常用符号和emoji, 全角标点转半角）
    4. 标准化空白字符

    Args:
        text (str): 待清洗的文本。
    Returns:
        str: 清洗后的文本。
    """
    # 替换所有空白符（含小红书常见的全角空格\u3000、换行、制表符）
    text = re.sub(r'[\t\n\u3000]', ' ', text)
    # HTML标签移除
    text = re.sub(r'<[^>]+>', '', text)
    # URL移除, 适配中文域名
    text = remove_urls(text)
    # 全角转半角（保留全角￥）
    text = fullwidth_to_halfwidth(text)
    # 符号标准化
    text = normalize_punctuation(text)
    # 标准化合并连续空格
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [42]:
# apply the basic_clean function to the 'text' column
cont['text'] = cont['text'].apply(basic_clean)

社交媒体特征处理层

In [ ]:
def social_media_clean(text):
    # 移除话题标签但保留关键词（如 #达拉斯美食 → 达拉斯美食）
    text = re.sub(r'#([^\s#]+)', r'\1', text)
    
    # 移除@提及
    text = re.sub(r'@\S+', '', text)
    
    # 转换表情符号（可选策略）
    strategy = 'remove'  # 或 'demojize'
    if strategy == 'remove':
        text = emoji.replace_emoji(text, replace='')
    else:
        text = emoji.demojize(text, delimiters=(' [', '] '))
    
    # 处理小红书特有格式（如[笑哭R]）
    text = re.sub(r'\[.*?\]', '', text)
    
    return text

'有没有和我一样【吃茶三千】一喝一个不吱声的 连地址都不想写了 虽然环境还不错，颜值也不错 但我感觉自己完全拔草了 尝了mango茶sample：仿佛化掉的芒果冰棍 冻顶乌龙：茶味白开水 又见柠檬挞：矮子里的将军，青柠汁绿茶+cream，虽然算清新派里能喝的，但依旧偏寡淡 松针厚奶：茶香精味重+依旧白开水 \t 是今天员工问题？还是达拉斯这家店问题？还是吃茶三千就这个风格？为什么值得排这么久的队呢…想不通啊想不通～'

In [ ]:
# 合并标题与正文
def process_content(note):
    return note['title'] + ' ' + note['note_body']

# 示例处理第一条数据
with open('contents_cooked.json') as f:
    notes = json.load(f)

sample_note = notes[0]
combined_text = process_content(sample_note)

# 清洗文本
def clean_text(text):
    text = re.sub(r'#\S+', '', text)  # 移除话题标签
    text = re.sub(r'\[.*?\]', '', text)  # 移除表情符号标记
    text = re.sub(r'\n', ' ', text)     # 换行符转空格
    return text.strip()

cleaned_text = clean_text(combined_text)

In [ ]:
# enable jieba , and use the accurate mode 
jieba.enable_paddle()
jieba.set_dictionary('..\..\Data\processed\dict.txt.big')
jieba.initialize()